<a href="https://colab.research.google.com/github/grendelaglaeca/XGBoost-brain-regions/blob/main/xgboost_tutorial_Chris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preliminaries**

Let’s first load the required packages. For any packages that you haven’t yet installed on your system, you can install them one at a time using the function install.packages('packagename').

In [17]:
import timeit

import numpy as np
import pandas as pd
import xgboost as xgb
import statsmodels.api as sm
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline

Load the data from a CSV file


In [18]:
url = "https://github.com/grendelaglaeca/XGBoost-brain-regions/raw/main/tutorial_all_data_speech.xlsx"
data = pd.read_excel(url)
print(data.shape)
data.head(20)

(2452, 96)


,Filename,Vocality,Emotion,Intensity,Statement,Repetition,Speaker,Sex,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0.2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,loudness_sma3_amean,loudness_sma3_stddevNorm,loudness_sma3_percentile20.0,loudness_sma3_percentile50.0,loudness_sma3_percentile80.0,loudness_sma3_pctlrange0.2,loudness_sma3_meanRisingSlope,loudness_sma3_stddevRisingSlope,loudness_sma3_meanFallingSlope,loudness_sma3_stddevFallingSlope,spectralFlux_sma3_amean,spectralFlux_sma3_stddevNorm,mfcc1_sma3_amean,mfcc1_sma3_stddevNorm,mfcc2_sma3_amean,mfcc2_sma3_stddevNorm,mfcc3_sma3_amean,mfcc3_sma3_stddevNorm,mfcc4_sma3_amean,mfcc4_sma3_stddevNorm,jitterLocal_sma3nz_amean,jitterLocal_sma3nz_stddevNorm,...,F2bandwidth_sma3nz_amean,F2bandwidth_sma3nz_stddevNorm,F2amplitudeLogRelF0_sma3nz_amean,F2amplitudeLogRelF0_sma3nz_stddevNorm,F3frequency_sma3nz_amean,F3frequency_sma3nz_stddevNorm,F3bandwidth_sma3nz_amean,F3bandwidth_sma3nz_stddevNorm,F3amplitudeLogRelF0_sma3nz_amean,F3amplitudeLogRelF0_sma3nz_stddevNorm,alphaRatioV_sma3nz_amean,alphaRatioV_sma3nz_stddevNorm,hammarbergIndexV_sma3nz_amean,hammarbergIndexV_sma3nz_stddevNorm,slopeV0.500_sma3nz_amean,slopeV0.500_sma3nz_stddevNorm,slopeV500.1500_sma3nz_amean,slopeV500.1500_sma3nz_stddevNorm,spectralFluxV_sma3nz_amean,spectralFluxV_sma3nz_stddevNorm,mfcc1V_sma3nz_amean,mfcc1V_sma3nz_stddevNorm,mfcc2V_sma3nz_amean,mfcc2V_sma3nz_stddevNorm,mfcc3V_sma3nz_amean,mfcc3V_sma3nz_stddevNorm,mfcc4V_sma3nz_amean,mfcc4V_sma3nz_stddevNorm,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0.500_sma3nz_amean,slopeUV500.1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,03-01-01-01-01-01-01,1,1,1,1,1,1,m,24.263884,0.104171,21.653511,23.907380,27.101028,5.447517,104.436584,113.437210,31.389978,8.258454,0.110051,1.410790,0.002538,0.011108,0.232714,0.230176,3.259262,1.573792,3.938445,2.248503,0.036235,1.529361,10.512092,1.711909,6.101207,1.387863,8.011750,1.559555,5.513017,1.962519,0.037905,0.945639,...,757.23315,0.307361,-141.36519,-0.603904,2510.8264,0.139656,654.06250,0.273573,-143.59050,-0.571859,-11.643292,-0.545192,19.907448,0.372820,0.073546,0.392833,-0.010039,-1.203686,0.103487,0.500103,29.751476,0.456312,4.090492,2.504729,17.560534,0.864205,5.178138,3.117834,-2.232880,10.586953,0.058064,0.011220,0.005713,2.127660,0.925926,0.336667,0.226912,0.540000,0.480260,-47.939644
1,03-01-01-01-01-02-01,1,1,1,1,2,1,m,25.167889,0.103561,23.178247,25.003677,27.699860,4.521612,56.695114,18.043581,9.703053,9.479507,0.113809,1.404945,0.003649,0.014939,0.233169,0.229521,3.134050,1.376852,3.778271,2.328364,0.036783,1.546615,10.702036,1.607827,5.343505,1.443376,10.708869,1.101803,5.429084,2.101397,0.023066,0.832704,...,737.92950,0.274377,-140.79219,-0.600151,2538.4785,0.134095,647.21800,0.340684,-142.90822,-0.570000,-12.260626,-0.531146,19.767748,0.371273,0.076232,0.362703,-0.009699,-1.099891,0.104238,0.542292,27.710243,0.479786,6.162842,1.679617,19.810240,0.690686,2.662904,6.122746,-2.234647,10.659966,0.050419,0.012741,0.005434,2.409639,1.223242,0.257500,0.089826,0.535000,0.485206,-47.342330
2,03-01-01-01-02-01-01,1,1,1,2,1,1,m,24.186604,0.103495,22.353912,23.728384,27.093287,4.739374,209.152560,255.810780,13.067563,11.153259,0.129210,1.429387,0.003478,0.009519,0.265525,0.262047,3.973667,2.611293,3.476154,2.330339,0.044583,1.494504,11.385279,1.617252,6.079310,1.649735,9.165839,1.211975,4.260278,2.831580,0.045453,1.859762,...,825.92786,0.285904,-135.41458,-0.652588,2462.3523,0.113135,702.81915,0.39904

**Features**

It will be helpful for us to store the names of the acoustic features so that we can access them more easily later on. We know from the summary above that the acoustic features begin at the ninth column in the data set and continue until the very last column, so we can simply extract those column names and verify that we’ve got the right ones:


In [19]:
col_names = list(data)  #gets a list of column headers
features = col_names[8:len(col_names)]  #python starts the count from zero
features

['F0semitoneFrom27.5Hz_sma3nz_amean',
 'F0semitoneFrom27.5Hz_sma3nz_stddevNorm',
 'F0semitoneFrom27.5Hz_sma3nz_percentile20.0',
 'F0semitoneFrom27.5Hz_sma3nz_percentile50.0',
 'F0semitoneFrom27.5Hz_sma3nz_percentile80.0',
 'F0semitoneFrom27.5Hz_sma3nz_pctlrange0.2',
 'F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope',
 'loudness_sma3_amean',
 'loudness_sma3_stddevNorm',
 'loudness_sma3_percentile20.0',
 'loudness_sma3_percentile50.0',
 'loudness_sma3_percentile80.0',
 'loudness_sma3_pctlrange0.2',
 'loudness_sma3_meanRisingSlope',
 'loudness_sma3_stddevRisingSlope',
 'loudness_sma3_meanFallingSlope',
 'loudness_sma3_stddevFallingSlope',
 'spectralFlux_sma3_amean',
 'spectralFlux_sma3_stddevNorm',
 'mfcc1_sma3_amean',
 'mfcc1_sma3_stddevNorm',
 'mfcc2_sma3_amean',
 'mfcc2_sma3_stddevNorm',
 'mfcc3_sma3_amean',
 'mfcc3_sma3_stddevNorm',
 'mfcc4_

These acoustic features were generated with openSMILE, using the GeMAPS feature set (v02). Specifically, these are the high-level, derived parameters (“functionals”) which make up the extended Geneva Minimalistic Acoustic Parameter Set (eGeMAPS). There are 88 parameters per audio file in the data base, which is exactly how many feature names are in the extracted *features* object:

# **XGBoost**
**preparation**

We’ll start by making a copy of the data set (so that anything we add/subtract/change doesn’t mess up the original data):

In [20]:
xgbdata = data

We’re going to set up an XGBoost model as a classification problem (i.e. is it sung or spoken?). For classification problems, XGBoost wants to have ordinal classes starting with 0. Let’s remind ourselves what our current class labels are:

In [21]:
xgbdata.Vocality.value_counts()   #equivalent of R *table* function

1    1440
2    1012
Name: Vocality, dtype: int64

We have an ordinal series (1: spoken, 2: sung) but it starts at 1, so we simply need to subtract 1 from the class series. We’ll put that into a separate column called *levels*:

In [22]:
xgbdata.Vocality

0       1
1       1
2       1
3       1
4       1
       ..
2447    2
2448    2
2449    2
2450    2
2451    2
Name: Vocality, Length: 2452, dtype: int64

In [23]:
xgbdata["levels"] = xgbdata.Vocality-1

In [24]:
xgbdata.levels.value_counts()

0    1440
1    1012
Name: levels, dtype: int64

Nice! Our classes are now ready for XGBoost classification.

One last thing before we start. Many types of calculations and algorithms will contain a random component of some sort. It’s always good practice to ensure that the random components can be replicated (either by yourself or by other researchers), so that your results will always come out the same way in the end. To do this in R, we choose some number (literally, any number!) and then set that as the random “seed” for the session. We’ll use the number 123:

In [25]:
random.seed(10)
print(random.random())

0.5714025946899135


**partioning data**


In machine learning of any kind, the general approach you want to take is to build a model that trains on a set of data, while at the same time validating what it has learned on a different set of data. Once you have a model that is tuned in this way, you then test the model on a third set of data. Importantly, these three sets should not contain the same data. This will lead to model overfit, which essentially means that the model has learned the patterns of the data too well, at the expense of any other patterns that might exist in the real world. Why is this a bad thing? A model that suffers from data overfitting will not generalize well to new data that it has never seen before… and, ultimately, generalizing to new data (i.e. being able to make accurate predictions from new data) is the entire point of machine learning in the first place!

An alternative to splitting the data into three parts (i.e. training, validation, testing) is to use an approach called cross-fold validation. In this approach, only two data partitions are made: training and testing. The training set is split up multiple times, and each time a portion of the training data is set aside for validation. The validation error is averaged across all of the iterations and is used as an indication of how well the model performs. Cross-fold validation is often used for small data sets (like ours!), where it is important to include as much training data as possible to build the model.

For our purposes, we will uses five folds in the cross-validation. This means that the training data will be split five separate times, and in each iteration 4/5 of the data (80%) will be used for training and 1/5 (20%) will be used for validation. This is not a hard-and-fast rule (70%-30% and 75%-25% splits are also used often), but an 80%-20% split is a nice way to ensure that we retain a lot of data for model training.

We’ll also mirror the 80%-20% in our initial selection of the training and testing data. What this means is that we will randomly select 80% of the data to perform all of the model tuning and set aside 20% of the data until the very end. To avoid model overfitting, we will test the model performance on the 20% that we set aside (the “hold-out” data) only one single time. This final performance will then give us the most reliable estimate of how well the model will then generalize to brand new data that it has never seen before.

Let’s first randomly select 80% of the data to use to build the model. We’ll use the row numbers to index which ones were selected:

In [37]:
X = xgbdata    # .drop('levels', axis=1) this will be extracted later
y = xgbdata['levels'].values
y = y.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

The final thing we need to do is construct two *xgb.DMatrix* objects, which are simply pointers that let XGBoost know which data and labels to associate with which data sets. Since we’ve already stored the acoustic features as the features variable, we can use that to select the proper columns. We’ll also create a variable *numberOfClasses*, which we will need later on:

In [38]:
test_matrix = xgb.DMatrix(data=X_test[features],
                          label=y_test)

train_matrix = xgb.DMatrix(data=X_train[features],
                          label=y_train)


n_classes = len(set(xgbdata.levels))

**hyperparameter tuning**

There are many different parameters that will affect the model architecture and, thus, how well the model performs. These are called hyperparameters. The default values for the hyperparameters in XGBoost generally perform very well, but it is possible to get even better performance by tuning the hyperparameters to your particular data set. The problem is that many of these hyperparameters interact with each other, so you will often need to change multiple hyperparameters to find the optimum values that yield the best model architecture.

One way of doing this is to search through all possible permutations of the different parameter values. This approach is known as a hyperparameter grid search. As you can imagine, grid searching can take a long time if you search through all possible permutations of all possible hyperparameter values! We won’t tune all of the hyperparameters here, but I encourage you to explore all of the XGBoost hyperparameters to see if there are others you want to try to implement in order to improve your model even further.

We will focus on only two here: max_depth and eta. max_depth will control how complex (i.e. deep) the tree model is; increasing the value will let the model learn more accurately but will also make it more likely to overfit. We will search all values between 3 and 15, in increments of 1 (NB: the default value is 6). eta is the “learning rate” of the model; increasing the value acts as a way of “shrinking” how well the model learns and will help guard against model overfitting. We will search all values between 0 and 0.4 in increments of 0.01 (NB: the default value is 0.3).

In [42]:
hyper_grid = {
    'max_depth': np.linspace(3, 15, 1),
    'eta': np.linspace(0, 0.4, 0.01)
}

len(hyper_grid)

TypeError: ignored